# Get Bounding Boxes

## Get coordinates, build dataframe, crop images

In [ ]:
!mkdir /kaggle/working/handcropped

rows = [[]]
index = 0
#bounding boxes were drawn in blue-green using MS Paint
myColor = [0,255,255]
myColor = np.array(myColor, dtype = 'uint8')

path = 'path_to_data'
folder = os.listdir(path)
for img in folder:
    image = os.path.join(path, img)
    image = cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    #eliminate all pixels that are not blue-green
    mask = cv2.inRange(image, myColor, myColor)
    output = cv2.bitwise_and(image, image, mask = mask)
    gray = cv2.cvtColor(output, cv2.COLOR_RGB2GRAY)
    ret, thresh = cv2.threshold(gray, 50, 255, 0)
    
    #get contours (corner coordinates) of bounding box
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    x = 0
    y = 0
    
    #get each corner
    for botright in contours:
        if botright[0][0][0] >= x:
            x2 = x = botright[0][0][0]
        if botright[0][0][1] >= y:
            y2 = y = botright[0][0][1]
    botright = [x,y]
    x = 100000
    y = 0
    for botleft in contours:
        if botleft[0][0][0] <= x:
            x = botleft[0][0][0]
        if botleft[0][0][1] >= y:
            y = botleft[0][0][1]
    botleft = [x,y]
    x = 100000
    y = 100000
    for topleft in contours:
        if topleft[0][0][0] <= x:
            x1 = x = topleft[0][0][0]
        if topleft[0][0][1] <= y:
            y1 = y = topleft[0][0][1]
    topleft = [x,y]
    x = 0
    y = 100000
    for topright in contours:
        if topright[0][0][0] >= x:
            x = topright[0][0][0]
        if topright[0][0][1] <= y:
            y = topright[0][0][1]
    topright = [x,y]
    #diagnostic check
    print('TL: {}, TR: {}, BL: {}, BR: {}'.format(topleft, topright, botleft, botright))
    
    #image naming conventions bad, ignore
    img = img.replace(' - BOUNDED', '')
    imgname = img
    
    #diagnostic check
    print(imgname, x1, y1, x2, y2)
    
    #add to dataframe
    rows.append([imgname, x1, y1, x2, y2])
    
    #crop image to bounding box and save
    cropped_image = image[topleft[1]+10:botleft[1]-10, topleft[0]+10:topright[0]-10]
    imgname = os.path.join('/kaggle/working/handcropped', img)
    try:
        cv2.imwrite(imgname, cropped_image)
    except Exception:
        print('error')

## Create CSV to download

In [ ]:
fields = ['ImageName', 'x1', 'y1', 'x2', 'y2']

filename = 'boundingboxes.csv'


with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(rows)

## Alternate Excel File

In [ ]:
import openpyxl

# Create a new workbook and select the active sheet
workbook = openpyxl.Workbook()
worksheet = workbook.active

# Define the fields and write the header row
fields = ['ImageName', 'x1', 'y1', 'x2', 'y2']
worksheet.append(fields)


for row in rows:
    worksheet.append(row)

# Save the workbook as an Excel file
filename = 'boundingboxes.xlsx'
workbook.save(filename)

## Compress cropped images for download

In [ ]:
!zip /kaggle/working/handcropped/handcropped.zip -r /kaggle/working/handcropped

## Display Sample of Cropped Images

In [ ]:
import os
import cv2
from random import sample
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (20,20))
croppedfolder = os.listdir('/kaggle/working/handcropped')
rows = 2
columns = 2
index = 1

for i, img in sample(list(enumerate(sorted(croppedfolder))), 4):
    img = os.path.join('/kaggle/working/handcropped', img)
    image = cv2.imread(img)
    fig.add_subplot(rows, columns, index)
    plt.imshow(image)
    plt.axis('off')
    plt.title('{} - {} down'.format(img, i))
    print(img)
    index += 1

# Augment Data

## Import

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
import mimetypes
import argparse
#import imutils
import random
from random import sample
import albumentations as A
from PIL import Image

random.seed(777)

## Augmentation Functions

In [ ]:
def hflip_with_default_label(image, bboxes):
    if not bboxes:
        return image, []

    labels = ['box'] * len(bboxes)
    transform = A.Compose([
        A.HorizontalFlip(p=1.0),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

    transformed = transform(image=image, bboxes=bboxes, labels=labels)
    return transformed['image'], transformed['bboxes']


def vflip_with_default_label(image, bboxes):
    if not bboxes:
        return image, []

    labels = ['box'] * len(bboxes)
    transform = A.Compose([
        A.VerticalFlip(p=1.0),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

    transformed = transform(image=image, bboxes=bboxes, labels=labels)
    return transformed['image'], transformed['bboxes']

def blur_with_default_label(image, bboxes):
    if not bboxes:
        return image, []

    labels = ['box'] * len(bboxes)
    transform = A.Compose([
        A.GaussianBlur (blur_limit=(7, 7), sigma_limit=0, always_apply=True, p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

    transformed = transform(image=image, bboxes=bboxes, labels=labels)
    return transformed['image'], transformed['bboxes']


def opticaldistort_with_default_label(image, bboxes):
    if not bboxes:
        return image, []

    labels = ['box'] * len(bboxes)
    transform = A.Compose([
        A.OpticalDistortion (distort_limit=0.35, shift_limit=0.35, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=True, p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

    transformed = transform(image=image, bboxes=bboxes, labels=labels)
    return transformed['image'], transformed['bboxes']

def rot90_with_default_label(image, bboxes):
    if not bboxes:
        return image, []

    labels = ['box'] * len(bboxes)
    transform = A.Compose([
        A.RandomRotate90(p=1.0),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

    transformed = transform(image=image, bboxes=bboxes, labels=labels)
    return transformed['image'], transformed['bboxes']

def randomscale_with_default_label(image, bboxes):
    if not bboxes:
        return image, []

    labels = ['box'] * len(bboxes)
    transform = A.Compose([
        A.RandomScale(scale_limit=0.35, interpolation=1, always_apply=True, p=1.0),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

    transformed = transform(image=image, bboxes=bboxes, labels=labels)
    return transformed['image'], transformed['bboxes']

def mix_with_default_label(image, bboxes):
    if not bboxes:
        return image, []

    labels = ['box'] * len(bboxes)
    transform = A.Compose([
        A.HorizontalFlip(p=1.0),
        A.VerticalFlip(p=1.0),
        A.GaussianBlur (blur_limit=(7, 7), sigma_limit=0, always_apply=True, p=1.0),
        A.OpticalDistortion (distort_limit=0.35, shift_limit=0.35, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=True, p=1.0),
        A.RandomRotate90(p=1.0),
        A.RandomScale(scale_limit=0.35, interpolation=1, always_apply=True, p=1.0),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

    transformed = transform(image=image, bboxes=bboxes, labels=labels)
    return transformed['image'], transformed['bboxes']

In [ ]:
df_path = "data_path"
img_path = "img_path"
df = load_workbook(df_path)
df = df.active
rows = [[]]
data = []
targets = []
filenames = []
labels = []
idx = 1

In [ ]:
for f in range(2,df.max_row):
    rows.append([df['A'+str(f)].value,df['B'+str(f)].value, df['C'+str(f)].value,df['D'+str(f)].value, df['E'+str(f)].value])
rows = rows[1:]

In [ ]:
!mkdir /kaggle/working/verticalflippedimages
!mkdir /kaggle/working/horizontalflippedimages
!mkdir /kaggle/working/blurredimages
!mkdir /kaggle/working/distortedimages
!mkdir /kaggle/working/rotatedimages
!mkdir /kaggle/working/scaledimages
!mkdir /kaggle/working/mixedimages

In [ ]:
import csv
fields = ['ImageName', 'Topleft', 'TopRight', 'BottomLeft', 'BottomRight']

csvname = 'newtag_boundaries.csv'
with open(csvname, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    for i, f in enumerate(rows):
        print('{}/{}'.format(i+1, len(rows)))
        print(f)
        filename = f[0]
        startx = f[1]
        starty = f[2]
        endx = f[3]
        endy = f[4]
        filenames.append(filename)

        imgpath = os.path.join(img_path, filename.replace('jpg', 'JPG'))
        print(imgpath)
        try:
            image = cv2.imread(imgpath)
            (h, w) = image.shape[:2]
            bbox = [startx, starty, endx, endy]
            bbox = [[int(coord) for coord in bbox]]
            targets.append((startx, starty, endx, endy))

            vflipimg, vflipbox = vflip_with_default_label(image=image, bboxes=bbox)
            hflipimg, hflipbox = hflip_with_default_label(image = image, bboxes = bbox)
            blurimg, blurbox = blur_with_default_label(image = image, bboxes = bbox)
            distortimg, distortbox = opticaldistort_with_default_label(image = image, bboxes = bbox)
            rotimg, rotbox = rot90_with_default_label(image = image, bboxes = bbox)
            scaleimg, scalebox = randomscale_with_default_label(image = image, bboxes = bbox)
            miximg, mixbox = mix_with_default_label(image = image, bboxes = bbox)
            vflipbox = ['{} - VFLIP.jpg'.format(filename.replace('.jpg', '').replace(' - ORIGINAL', '').replace('.JPG', '')), vflipbox[0][0], vflipbox[0][1], vflipbox[0][2], vflipbox[0][3]]
            hflipbox = ['{} - HFLIP.jpg'.format(filename.replace('.jpg', '').replace(' - ORIGINAL', '').replace('.JPG', '')), hflipbox[0][0], hflipbox[0][1], hflipbox[0][2], hflipbox[0][3]]
            blurbox = ['{} - BLURRED.jpg'.format(filename.replace('.jpg', '').replace(' - ORIGINAL', '').replace('.JPG', '')), blurbox[0][0], blurbox[0][1], blurbox[0][2], blurbox[0][3]]
            distortbox = ['{} - DISTORTED.jpg'.format(filename.replace('.jpg', '').replace(' - ORIGINAL', '').replace('.JPG', '')), distortbox[0][0], distortbox[0][1], distortbox[0][2], distortbox[0][3]]
            rotbox = ['{} - ROTATED.jpg'.format(filename.replace('.jpg', '').replace('.JPG', '')), int(rotbox[0][0]), int(rotbox[0][1]), int(rotbox[0][2]), int(rotbox[0][3])]
            scalebox = ['{} - SCALED.jpg'.format(filename.replace('.jpg', '').replace('.JPG', '')), int(scalebox[0][0]), int(scalebox[0][1]), int(scalebox[0][2]), int(scalebox[0][3])]
            mixbox = ['{} - MIXED.jpg'.format(filename.replace('.jpg', '').replace('.JPG', '')), int(mixbox[0][0]), int(mixbox[0][1]), int(mixbox[0][2]), int(mixbox[0][3])]
            bbox = [f[0], bbox[0][0], bbox[0][1], bbox[0][2], bbox[0][3]]
            csvwriter.writerow(bbox)
            csvwriter.writerow(vflipbox)
            csvwriter.writerow(hflipbox)
            csvwriter.writerow(blurbox)
            csvwriter.writerow(distortbox)
            csvwriter.writerow(rotbox)
            csvwriter.writerow(scalebox)
            csvwriter.writerow(mixbox)
            cv2.imwrite(os.path.join('/kaggle/working/verticalflippedimages', f[0].replace('.jpg', '').replace('.JPG', '') + ' - VFLIP.jpg'), vflipimg)
            cv2.imwrite(os.path.join('/kaggle/working/horizontalflippedimages', f[0].replace('.jpg', '').replace('.JPG', '') + ' - HFLIP.jpg'), hflipimg)
            cv2.imwrite(os.path.join('/kaggle/working/blurredimages', f[0].replace('.jpg', '').replace('.JPG', '') + ' - BLURRED.jpg'), blurimg)
            cv2.imwrite(os.path.join('/kaggle/working/distortedimages', f[0].replace('.jpg', '').replace('.JPG', '') + ' - DISTORTED.jpg'), distortimg)
            cv2.imwrite(os.path.join('/kaggle/working/rotatedimages', f[0].replace('.jpg', '').replace('.JPG', '') + ' - ROTATED.jpg'), rotimg)
            try:
                cv2.imwrite(os.path.join('/kaggle/working/scaledimages', f[0].replace('.jpg', '').replace('.JPG', '') + ' - SCALED.jpg'), scaleimg)
            except:
                print('error in save scaled')
            print(' THE LOOP WORKED')
            cv2.imwrite(os.path.join('/kaggle/working/mixedimages', f[0].replace('.jpg', '').replace('.JPG', '') + ' - MIXED.jpg'), miximg)
        except:
            print('File not Found:', imgpath)

In [ ]:
!zip /kaggle/working/augmented.zip -r /kaggle/working/

In [ ]:
import csv
fields = ['ImageName', 'TopLeft', 'TopRight', 'BottomLeft', 'BottomRight']

filename = 'newtag_boundaries.csv'

with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(vflip_boxes)
    for row, image, box in zip(rows, vflip_imgs, vflip_boxes):
        cv2.imwrite(os.path.join('/kaggle/working/verticalflippedimages', row[0]), image)

# Train Tag-Cropping Model

## Import

In [ ]:
 
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os
from tensorflow.keras.models import load_model
import mimetypes
import argparse
import random
from random import sample
import albumentations as A
from PIL import Image

random.seed(42)

## Create Tag Image Folder

In [ ]:
import shutil

source_folder1 = '/kaggle/input/augmented/Images'
source_folder2 = '/kaggle/input/augmented-images-2/Images 2'
source_folder3 = '/kaggle/input/1016-augmented/Images 3'
destination_folder = '/kaggle/working/Images'

os.makedirs(destination_folder, exist_ok=True)

for i, file_name in enumerate(os.listdir(source_folder1)):
    print('Progress: {}/{}'.format(i, len(os.listdir(source_folder1))), end = '\r')
    shutil.copy2(os.path.join(source_folder1, file_name), destination_folder)

for o, file_name in enumerate(os.listdir(source_folder2)):
    print('Progress: {}/{}'.format(o, len(os.listdir(source_folder2))), end = '\r')
    shutil.copy2(os.path.join(source_folder2, file_name), destination_folder)


for u, file_name in enumerate(os.listdir(source_folder3)):
    print('Progress: {}/{}'.format(u, len(os.listdir(source_folder3))), end = '\r')
    shutil.copy2(os.path.join(source_folder3, file_name), destination_folder)

In [ ]:
df_path = "/kaggle/input/1016-augmented/augmented_boundary_master.xlsx"
img_path = "/kaggle/working/Images"
df = load_workbook(df_path)
df = df.active
rows = [[]]
data = []
targets = []
filenames = []
trynames = []
labels = []
idx = 1

## Data Cleaning/Separation

In [ ]:
for f in range(2,df.max_row):
    tryname = df['A'+str(f)].value
    augmented = 0
    for effect in ['VFLIP', 'HFLIP', 'DISTORTED', 'MIXED', 'ROTATED', 'SCALED', 'BLURRED']:
        if effect in tryname:
            augmented = 1
            rows.append([df['A'+str(f)].value,df['B'+str(f)].value, df['C'+str(f)].value,df['D'+str(f)].value, df['E'+str(f)].value] )
            #if tryname in os.listdir('/kaggle/input/cropped-images/1003 cropped images') or tryname.replace('jpg', 'JPG') in os.listdir('/kaggle/input/1016-augmented/toAugment'):
    if augmented != 1:
        trynames.append([df['A'+str(f)].value,df['B'+str(f)].value, df['C'+str(f)].value,df['D'+str(f)].value, df['E'+str(f)].value] )
rows = rows[1:]
print(len(rows))


In [ ]:
random.shuffle(rows)
random.shuffle(trynames)

In [ ]:
imgnames = []
bboxcoords = []
trans_imgs = []
trans_imgs = []
trans_boxes = []
#print(len(rows))
for i, f in enumerate(rows):
    print('progress: {}/{} *** '.format(i, len(rows)), end = '\r')
    # print(f[0])
    filename = f[0]
    if filename not in os.listdir('/kaggle/working/Images'):
        print('not in here')
        continue
    if '0164' in filename:
        filename = filename.replace('site_1', 'site_2').replace('002','001')
        pass
    #print(filename)
    startx = int(f[1])
    starty = int(f[2])
    endx = int(f[3])
    endy = int(f[4])
    imgnames.append(filename)
    try:
        imgpath = os.path.join(img_path, filename)

        image = cv2.imread(imgpath)
        #1
        (h, w) = image.shape[:2]
        bbox = [startx, starty, endx, endy]

        bbox = [[int(coord) for coord in bbox]]
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except AttributeError:
        try:
            print('trying with - BOUNDED added')
            try:
                imgpath = os.path.join(img_path, filename).replace('.jpg', ' - BOUNDED.jpg')

                image = cv2.imread(imgpath)
                #2
                (h, w) = image.shape[:2]
                bbox = [startx, starty, endx, endy]

                bbox = [[int(coord) for coord in bbox]]
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            except AttributeError:
                try:
                    print('trying .JPG')
                    imgpath = os.path.join(img_path, filename).replace('.jpg', '.JPG')

                    image = cv2.imread(imgpath)
                    #3
                    (h, w) = image.shape[:2]
                    bbox = [startx, starty, endx, endy]

                    bbox = [[int(coord) for coord in bbox]]
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                except FileNotFoundError or AttributeError:
                    continue
        except FileNotFoundError or AttributeError:
            continue
    hflipimg, hflipbox = hflip_with_default_label(image=image, bboxes=bbox)
    trans_imgs.append(hflipimg)
    trans_boxes.append(hflipbox)
    vflipimg, vflipbox = vflip_with_default_label(image=image, bboxes=bbox)
    trans_imgs.append(vflipimg)
    trans_boxes.append(vflipbox)
    blurimg, blurbox = blur_with_default_label(image=image, bboxes=bbox)
    trans_imgs.append(blurimg)
    trans_boxes.append(blurbox)

    distortimg, distortbox = opticaldistort_with_default_label(image=image, bboxes=bbox)
    trans_imgs.append(distortimg)
    trans_boxes.append(distortbox)
    startx = float(startx) / w
    starty = float(starty) / h
    endx = float(endx) / w
    endy = float(endy) / h
    print('X1: {}, y1: {}, x2: {}, y2: {}'.format(startx, starty, endx, endy),end = '\r')
    image = load_img(imgpath, target_size=(224, 224))
    image = img_to_array(image)
    data.append(image)
    targets.append((startx, starty, endx, endy))
    filenames.append(filename)
print(len(trans_imgs), len(trans_boxes))
print(len(data), len(targets))

In [ ]:
traindata = []
traintargets = []
names = []
errors = 0

for i, f in enumerate(trynames):
    name = f[0]
    names.append(name)
    startx = int(f[1])
    starty = int(f[2])
    endx = int(f[3])
    endy = int(f[4])
    try:
        imgpath = os.path.join('/kaggle/working/Images', name)
        image = cv2.imread(imgpath)
        (h, w) = image.shape[:2]
        bbox = [startx, starty, endx, endy]
        bbox = [[int(coord) for coord in bbox]]
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    except (AttributeError, FileNotFoundError):
        try:
            print('trying with - BOUNDED added')
            imgpath = os.path.join(img_path, name).replace('.jpg', ' - BOUNDED.jpg')
            image = cv2.imread(imgpath)
            (h, w) = image.shape[:2]
            bbox = [startx, starty, endx, endy]
            bbox = [[int(coord) for coord in bbox]]
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        except (AttributeError, FileNotFoundError):
            print('trying .JPG')
            imgpath = os.path.join(img_path, name).replace('.jpg', '.JPG')
            image = cv2.imread(imgpath)
            if image is None:
                try:
                    imgpath = os.path.join('/kaggle/input/1016-augmented/toAugment', name.replace('jpg', 'JPG'))
                    image = cv2.imread(imgpath)
                    (h, w) = image.shape[:2]
                    bbox = [startx, starty, endx, endy]
                    bbox = [[int(coord) for coord in bbox]]
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                except (AttributeError, FileNotFoundError):
                    print(f"Skipping {name} due to an error.")
                    errors += 1
                    continue
            (h, w) = image.shape[:2]
            bbox = [startx, starty, endx, endy]
            bbox = [[int(coord) for coord in bbox]]
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    startx = float(startx) / w
    starty = float(starty) / h
    endx = float(endx) / w
    endy = float(endy) / h

    print('X1: {}, y1: {}, x2: {}, y2: {}'.format(startx, starty, endx, endy))

    image = load_img(imgpath, target_size=(224, 224))
    image = img_to_array(image)
    traindata.append(image)
    traintargets.append((startx, starty, endx, endy))

In [ ]:
def iou_calc(bb1, bb2):
    true_xmin, true_ymin, true_width, true_height = tf.split(bb1, 4, axis=-1)
    bb_xmin, bb_ymin, bb_width, bb_height = tf.split(bb2, 4, axis=-1)

    true_xmax = true_xmin + true_width
    true_ymax = true_ymin + true_height
    bb_xmax = bb_xmin + bb_width
    bb_ymax = bb_ymin + bb_height

    # calculating area
    true_area = true_width * true_height
    bb_area = bb_width * bb_height

    # calculating intersection coordinates
    inter_xmin = tf.maximum(true_xmin, bb_xmin)
    inter_ymin = tf.maximum(true_ymin, bb_ymin)
    inter_xmax = tf.minimum(true_xmax, bb_xmax)
    inter_ymax = tf.minimum(true_ymax, bb_ymax)

    inter_width = tf.maximum(0.0, inter_xmax - inter_xmin)
    inter_height = tf.maximum(0.0, inter_ymax - inter_ymin)
    inter_area = inter_width * inter_height

    iou = inter_area / (true_area + bb_area - inter_area)

    return iou

def iou_loss(y_true, y_pred):
    loss = 1 - iou_calc(y_true, y_pred)
    iou_check = tf.reduce_all(loss <= 1)
    tf.debugging.assert_scalar(iou_check, message="IOU values must be less than or equal to 1.")
    iou_check = tf.reduce_all(loss >= 0)
    tf.debugging.assert_scalar(iou_check, message="IOU values must be greater than or equal to 0.")
    return loss

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K

def calculate_ciou_loss(y_true, y_pred):
    # Extract the coordinates of the bounding boxes
    true_x, true_y, true_w, true_h = tf.unstack(y_true, 4, axis=-1)
    pred_x, pred_y, pred_w, pred_h = tf.unstack(y_pred, 4, axis=-1)

    # Calculate the intersection coordinates
    intersect_x1 = tf.maximum(true_x - true_w / 2.0, pred_x - pred_w / 2.0)
    intersect_y1 = tf.maximum(true_y - true_h / 2.0, pred_y - pred_h / 2.0)
    intersect_x2 = tf.minimum(true_x + true_w / 2.0, pred_x + pred_w / 2.0)
    intersect_y2 = tf.minimum(true_y + true_h / 2.0, pred_y + pred_h / 2.0)

    # Calculate the intersection area
    intersect_w = tf.maximum(0.0, intersect_x2 - intersect_x1)
    intersect_h = tf.maximum(0.0, intersect_y2 - intersect_y1)
    intersect_area = intersect_w * intersect_h
    print('INTERSECT AREA: {}'.format(intersect_area))

    # Calculate the union area
    true_area = true_w * true_h
    pred_area = pred_w * pred_h
    union_area = true_area + pred_area - intersect_area
    
    print('UNION AREA: {}'.format(union_area))

    # Calculate the IoU
    iou = intersect_area / tf.maximum(union_area, K.epsilon())
    print('IOU: {}'.format(iou))

    # Calculate the enclosing box coordinates
    enclose_x1 = tf.minimum(true_x - true_w / 2.0, pred_x - pred_w / 2.0)
    enclose_y1 = tf.minimum(true_y - true_h / 2.0, pred_y - pred_h / 2.0)
    enclose_x2 = tf.maximum(true_x + true_w / 2.0, pred_x + pred_w / 2.0)
    enclose_y2 = tf.maximum(true_y + true_h / 2.0, pred_y + pred_h / 2.0)

    # Calculate the diagonal distance squared
    enclose_w = tf.maximum(0.0, enclose_x2 - enclose_x1)
    enclose_h = tf.maximum(0.0, enclose_y2 - enclose_y1)
    diagonal_dist_sq = enclose_w**2 + enclose_h**2

    # Calculate the center distance squared
    center_dist_sq = (true_x - pred_x)**2 + (true_y - pred_y)**2

    # Calculate the CIoU loss
    ciou = iou - (center_dist_sq / diagonal_dist_sq)

    # Return the CIoU loss
    #return 1.0 - ciou
    return 1.0 - iou

def ciou_loss(y_true, y_pred):
    loss = calculate_ciou_loss(y_true, y_pred)
    return tf.reduce_mean(loss)

In [ ]:
data = np.array(data, dtype='float32') / 255.0
traindata = np.array(traindata, dtype='float32') / 255.0
targets = np.array(targets, dtype = 'float32')
traintargets = np.array(traintargets, dtype = 'float32')

In [ ]:
trainImages = data
testImages = traindata
trainTargets = targets
testTargets = traintargets
trainFilenames = filenames
testFilenames = names

In [ ]:
from tensorflow.keras.optimizers.schedules import CosineDecay

decay_steps = 300
initial_learning_rate = 0.001



lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps
)

In [ ]:
vgg = VGG16(weights = 'imagenet', include_top = False, input_tensor=Input(shape=(224,224,3)))

vgg.trainable = False

flatten = vgg.output
flatten = Flatten()(flatten)

bboxhead = Dense(256, activation = 'relu')(flatten)
bboxhead = Dense(128, activation = 'relu')(bboxhead)
bboxhead = Dense(64, activation = 'relu')(bboxhead)
bboxhead = Dense(32, activation = 'relu')(bboxhead)
bboxhead = Dense(4, activation = 'sigmoid')(bboxhead)

model = Model(inputs = vgg.input, outputs = bboxhead)

opt = Adam(lr = lr_schedule)
model.compile(loss =ciou_loss, optimizer = opt, metrics = [tfa.losses.GIoULoss(), 'mse'])

In [ ]:
lr_history = []

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_schedule_callback(epoch, lr):
    # Log the learning rate at each epoch
    print(f"Learning rate at epoch {epoch+1}: {lr}")
    lr_history.append(lr)
    return lr_schedule(epoch)  # U

In [ ]:
checkpoint_callback = ModelCheckpoint(filepath='/kaggle/working/best_model.h5',
                                      monitor='val_loss',
                                      save_best_only=True,
                                      mode='min',
                                      verbose=1)

In [ ]:
 earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=25,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=25
)

In [ ]:
trainlength = 300
H = model.fit(trainImages, trainTargets, validation_data = (testImages,testTargets), batch_size = 32, epochs = trainlength, verbose = 1, callbacks = [checkpoint_callback, LearningRateScheduler(lr_schedule_callback)])

In [ ]:
import plotly.graph_objects as go
import plotly.offline as offline

N = 300

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(0, N), y=H.history['loss'], name='train_loss'))
fig.add_trace(go.Scatter(x=np.arange(0, N), y=H.history['val_loss'], name='val_loss'))
fig.update_layout(title="Bounding Box Regression MSE Loss on Training Set",
                  xaxis_title="Epochs",
                  yaxis_title="Loss")
offline.plot(fig, filename='CIOU graph.html', auto_open=False)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(0, N), y=H.history['giou_loss'], name='train GIoU'))
fig.add_trace(go.Scatter(x=np.arange(0, N), y=H.history['val_giou_loss'], name='val GIoU'))
fig.update_layout(title="Bounding Box Regression GIoU Loss on Training Set",
                  xaxis_title="Epochs",
                  yaxis_title="Loss")
fig.show()

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("/kaggle/working/model 1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/kaggle/working/model 1.h5")

# Train ID-Cropping Model

In [ ]:
import shutil

source_folder1 = '/kaggle/input/augmented/Images'
source_folder2 = '/kaggle/input/augmented-images-2/Images 2'
source_folder3 = '/kaggle/input/1016-augmented/Images 3'
destination_folder = '/kaggle/working/Images'

os.makedirs(destination_folder, exist_ok=True)

for i, file_name in enumerate(os.listdir(source_folder1)):
    print('Progress: {}/{}'.format(i, len(os.listdir(source_folder1))), end = '\r')
    shutil.copy2(os.path.join(source_folder1, file_name), destination_folder)

for o, file_name in enumerate(os.listdir(source_folder2)):
    print('Progress: {}/{}'.format(o, len(os.listdir(source_folder2))), end = '\r')
    shutil.copy2(os.path.join(source_folder2, file_name), destination_folder)


for u, file_name in enumerate(os.listdir(source_folder3)):
    print('Progress: {}/{}'.format(u, len(os.listdir(source_folder3))), end = '\r')
    shutil.copy2(os.path.join(source_folder3, file_name), destination_folder)

In [ ]:
df_path = "/kaggle/input/1016-augmented/augmented_boundary_master.xlsx"
img_path = "/kaggle/working/Images"
df = load_workbook(df_path)
df = df.active
rows = [[]]
data = []
targets = []
filenames = []
trynames = []
labels = []
idx = 1

In [ ]:
for f in range(2,df.max_row):
    tryname = df['A'+str(f)].value
    augmented = 0
    for effect in ['VFLIP', 'HFLIP', 'DISTORTED', 'MIXED', 'ROTATED', 'SCALED', 'BLURRED']:
        if effect in tryname:
            augmented = 1
            rows.append([df['A'+str(f)].value,df['B'+str(f)].value, df['C'+str(f)].value,df['D'+str(f)].value, df['E'+str(f)].value] )
    if augmented != 1:
        trynames.append([df['A'+str(f)].value,df['B'+str(f)].value, df['C'+str(f)].value,df['D'+str(f)].value, df['E'+str(f)].value] )
rows = rows[1:]
print(len(rows))

In [ ]:
random.shuffle(rows)
random.shuffle(trynames)

In [ ]:
imgnames = []
bboxcoords = []
trans_imgs = []
trans_imgs = []
trans_boxes = []
for i, f in enumerate(rows):
    print('progress: {}/{} *** '.format(i, len(rows)), end = '\r')
    # print(f[0])
    filename = f[0]
    if filename not in os.listdir('/kaggle/working/Images'):
        print('not in here')
        continue
    if '0164' in filename:
        filename = filename.replace('ba6ham', 'cu2val').replace('002','001')
        pass
    startx = int(f[1])
    starty = int(f[2])
    endx = int(f[3])
    endy = int(f[4])
    imgnames.append(filename)
    try:
        imgpath = os.path.join(img_path, filename)

        image = cv2.imread(imgpath)
        (h, w) = image.shape[:2]
        bbox = [startx, starty, endx, endy]

        bbox = [[int(coord) for coord in bbox]]
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except AttributeError:
        try:
            print('trying with - BOUNDED added')
            try:
                imgpath = os.path.join(img_path, filename).replace('.jpg', ' - BOUNDED.jpg')

                image = cv2.imread(imgpath)
                (h, w) = image.shape[:2]
                bbox = [startx, starty, endx, endy]

                bbox = [[int(coord) for coord in bbox]]
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            except AttributeError:
                try:
                    print('trying .JPG')
                    imgpath = os.path.join(img_path, filename).replace('.jpg', '.JPG')

                    image = cv2.imread(imgpath)
                    (h, w) = image.shape[:2]
                    bbox = [startx, starty, endx, endy]

                    bbox = [[int(coord) for coord in bbox]]
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                except FileNotFoundError or AttributeError:
                    continue
        except FileNotFoundError or AttributeError:
            continue
    hflipimg, hflipbox = hflip_with_default_label(image=image, bboxes=bbox)
    trans_imgs.append(hflipimg)
    trans_boxes.append(hflipbox)
    vflipimg, vflipbox = vflip_with_default_label(image=image, bboxes=bbox)
    trans_imgs.append(vflipimg)
    trans_boxes.append(vflipbox)
    blurimg, blurbox = blur_with_default_label(image=image, bboxes=bbox)
    trans_imgs.append(blurimg)
    trans_boxes.append(blurbox)

    distortimg, distortbox = opticaldistort_with_default_label(image=image, bboxes=bbox)
    trans_imgs.append(distortimg)
    trans_boxes.append(distortbox)
    startx = float(startx) / w
    starty = float(starty) / h
    endx = float(endx) / w
    endy = float(endy) / h
    print('X1: {}, y1: {}, x2: {}, y2: {}'.format(startx, starty, endx, endy),end = '\r')
    image = load_img(imgpath, target_size=(224, 224))
    image = img_to_array(image)
    data.append(image)
    targets.append((startx, starty, endx, endy))
    filenames.append(filename)
print(len(trans_imgs), len(trans_boxes))
print(len(data), len(targets))

In [ ]:
traindata = []
traintargets = []
names = []
errors = 0

for i, f in enumerate(trynames):
    name = f[0]
    names.append(name)
    startx = int(f[1])
    starty = int(f[2])
    endx = int(f[3])
    endy = int(f[4])
    try:
        imgpath = os.path.join('/kaggle/working/Images', name)
        image = cv2.imread(imgpath)
        (h, w) = image.shape[:2]
        bbox = [startx, starty, endx, endy]
        bbox = [[int(coord) for coord in bbox]]
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    except (AttributeError, FileNotFoundError):
        try:
            print('trying with - BOUNDED added')
            imgpath = os.path.join(img_path, name).replace('.jpg', ' - BOUNDED.jpg')
            image = cv2.imread(imgpath)
            (h, w) = image.shape[:2]
            bbox = [startx, starty, endx, endy]
            bbox = [[int(coord) for coord in bbox]]
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        except (AttributeError, FileNotFoundError):
            print('trying .JPG')
            imgpath = os.path.join(img_path, name).replace('.jpg', '.JPG')
            image = cv2.imread(imgpath)
            if image is None:
                try:
                    imgpath = os.path.join('/kaggle/input/1016-augmented/toAugment', name.replace('jpg', 'JPG'))
                    image = cv2.imread(imgpath)
                    (h, w) = image.shape[:2]
                    bbox = [startx, starty, endx, endy]
                    bbox = [[int(coord) for coord in bbox]]
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                except (AttributeError, FileNotFoundError):
                    print(f"Skipping {name} due to an error.")
                    errors += 1
                    continue
            (h, w) = image.shape[:2]
            bbox = [startx, starty, endx, endy]
            bbox = [[int(coord) for coord in bbox]]
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    startx = float(startx) / w
    starty = float(starty) / h
    endx = float(endx) / w
    endy = float(endy) / h

    print('X1: {}, y1: {}, x2: {}, y2: {}'.format(startx, starty, endx, endy))

    image = load_img(imgpath, target_size=(224, 224))
    image = img_to_array(image)
    traindata.append(image)
    traintargets.append((startx, starty, endx, endy))


In [ ]:
data = np.array(data, dtype='float32') / 255.0
traindata = np.array(traindata, dtype='float32') / 255.0
targets = np.array(targets, dtype = 'float32')
traintargets = np.array(traintargets, dtype = 'float32')

In [ ]:
trainImages = data
testImages = traindata
trainTargets = targets
testTargets = traintargets
trainFilenames = filenames
testFilenames = names

In [ ]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers.schedules import CosineDecay

decay_steps = 300
initial_learning_rate = 0.0001



lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps
)

In [ ]:
base_model = VGG16(weights = 'imagenet', include_top = False, input_tensor=Input(shape=(224,224,3)))

freeze_until_layer = 7

for layer in base_model.layers[:freeze_until_layer]:
    layer.trainable = False

base_model.summary()

In [ ]:
flatten = base_model.output
flatten = Flatten()(flatten)


bboxhead = Dense(256, activation = 'relu')(flatten)
bboxhead = Dense(128, activation = 'relu')(bboxhead)
bboxhead = Dense(64, activation = 'relu')(bboxhead)
bboxhead = Dense(32, activation = 'relu')(bboxhead)
bboxhead = Dense(4, activation = 'sigmoid')(bboxhead)

model = Model(inputs = base_model.input, outputs = bboxhead)

opt = Adam(lr = lr_schedule)
model.compile(loss =ciou_loss, optimizer = opt, metrics = [tfa.losses.GIoULoss(), 'mse'])

In [ ]:
lr_history = []

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_schedule_callback(epoch, lr):
    # Log the learning rate at each epoch
    print(f"Learning rate at epoch {epoch+1}: {lr}")
    lr_history.append(lr)
    return lr_schedule(epoch)  # U

In [ ]:
checkpoint_callback = ModelCheckpoint(filepath='/kaggle/working/best_model.h5',
                                      monitor='val_loss',
                                      save_best_only=True,
                                      mode='min',
                                      verbose=1)

In [ ]:
 earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=25,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=25
)

In [ ]:
trainlength = 300
H = model.fit(trainImages, trainTargets, validation_data = (testImages,testTargets), batch_size = 32, epochs = trainlength, verbose = 1, callbacks = [checkpoint_callback, LearningRateScheduler(lr_schedule_callback)])

In [ ]:
import plotly.graph_objects as go
import plotly.offline as offline

N = 300

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(0, N), y=H.history['loss'], name='train_loss'))
fig.add_trace(go.Scatter(x=np.arange(0, N), y=H.history['val_loss'], name='val_loss'))
fig.update_layout(title="Bounding Box Regression CIOU Loss on Training Set",
                  xaxis_title="Epochs",
                  yaxis_title="Loss")
offline.plot(fig, filename='CIOU graph.html', auto_open=False)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(0, N), y=H.history['giou_loss'], name='train GIoU'))
fig.add_trace(go.Scatter(x=np.arange(0, N), y=H.history['val_giou_loss'], name='val GIoU'))
fig.update_layout(title="Bounding Box Regression GIoU Loss on Training Set",
                  xaxis_title="Epochs",
                  yaxis_title="Loss")
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(0, N), y=H.history['lr'], name='Learning Rate'))
fig.update_layout(title="Exponential Decay Learning Rate",
                  xaxis_title="Epochs",
                  yaxis_title="lr")
fig.show()

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("/kaggle/working/model 1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/kaggle/working/model 1.h5")

# Read the Tags

In [ ]:
!pip install imutils
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
import mimetypes
import argparse
import imutils
import random
from random import sample
import albumentations as A
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import numpy as np
import mimetypes
import argparse
import imutils
import cv2
import os
import shutil
from random import sample


random.seed(42)

In [ ]:
source_folder1 = '/kaggle/input/example-photoset/example'

destination_folder = '/kaggle/working/Images'

os.makedirs(destination_folder, exist_ok=True)

for i, file_name in enumerate(os.listdir(source_folder1)):
    print('Progress: {}/{}'.format(i, len(os.listdir(source_folder1))), end = '\r')
    shutil.copy2(os.path.join(source_folder1, file_name), destination_folder)

In [ ]:
tagmodel_json_file = open('/kaggle/input/tag-model/model 1.json', 'r')
tagmodel_loaded_model_json = tagmodel_json_file.read()
tagmodel_json_file.close()
tagmodel = model_from_json(tagmodel_loaded_model_json)
# load weights into new model
tagmodel.load_weights("/kaggle/input/tag-model/best_model.h5")

textmodel_json_file = open('/kaggle/input/getid-model/model 1.json', 'r')
textmodel_loaded_model_json = textmodel_json_file.read()
textmodel_json_file.close()
textmodel = model_from_json(textmodel_loaded_model_json)
# load weights into new model
textmodel.load_weights("/kaggle/input/getid-model/best_model.h5")

In [ ]:
!mkdir /kaggle/working/cropped

In [ ]:
imagepath = '/kaggle/input/example-photoset/example'

print(len(os.listdir(imagepath)))

from random import sample


for num in range(len(os.listdir(imagepath))):
    if num == 0 or num % 2 == 0:
        picture = sorted(os.listdir(imagepath))[num]
        pic = os.path.join(imagepath, picture)
        print(pic)
        image = load_img(pic, target_size = (224,224))
        image = img_to_array(image) / 255.0
        image = np.expand_dims(image, axis = 0)
        predtag = tagmodel.predict(image)[0]
        (startx, starty, endx, endy) = predtag

        image = cv2.imread(pic)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        (h, w) = image.shape[:2]
        startx = int(startx * w)
        starty = int(starty * h)
        endx = int(endx * w)
        endy = int(endy * h)
        image = image[int(starty):int(endy), int(startx):int(endx)]
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.imwrite('/kaggle/working/cropped/'+ picture, image)
    else:
        print('tree file', sorted(os.listdir(imagepath))[num])

In [ ]:
!mkdir /kaggle/working/textcropped

In [ ]:
imagepath = '/kaggle/working/cropped'

jpg_files = [img for img in os.listdir(imagepath)]

print(jpg_files)

for img in jpg_files:
    myimg = os.path.join(imagepath, img)
    image = load_img(myimg, target_size = (224,224))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis = 0)
    preds = textmodel.predict(image)[0]
    (startx, starty, endx, endy) = preds
    print(preds)

    image = cv2.imread(myimg)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = imutils.resize(image, width=image.shape[1])
    (h, w) = image.shape[:2]
    startx = int(startx * w)
    starty = int(starty * h)
    endx = int(endx * w)
    endy = int(endy * h)
    startx -= int(0.025*startx)
    starty -=int(0.025*starty)
    endx += int(0.025*endx)
    endy += int(0.025*endy)
    cv2.rectangle(image, (startx, starty), (endx, endy), (0, 0, 0), 5)
    image = image[int(starty):int(endy), int(startx):int(endx)]
    cv2.imwrite('/kaggle/working/textcropped/' + img, image)

In [ ]:
def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False
    
def remove_non_numeric(string):
    return ''.join(c for c in string if is_number(c))
def attempt_read(image, threshold, highestconf, threshoption):
    yourtag = 'NO READ'
    picture = cv2.imread(image)
    graycrop = cv2.cvtColor(picture, cv2.COLOR_BGR2RGB)
    graycrop = cv2.cvtColor(graycrop, cv2.COLOR_RGB2GRAY)

    kernel = np.ones((5,5), np.float32)/25
    graycrop = cv2.filter2D(graycrop, -1, kernel)
    ret, graycrop = cv2.threshold(graycrop, threshold, 255, cv2.THRESH_BINARY)
    th2 = cv2.adaptiveThreshold(graycrop,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
                cv2.THRESH_BINARY,11,2)
    th3 = cv2.adaptiveThreshold(graycrop,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                cv2.THRESH_BINARY,11,2)
    retotsu,thotsu = cv2.threshold(graycrop,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    mydict = {'graycrop': graycrop,'th2': th2, 'th3': th3, 'thotsu': thotsu}
    #if threshoption == 'graycrop':
        #plt.figure(figsize = (5,5))
        #plt.imshow(mydict[threshoption], cmap = 'gray')
        #plt.show()
    for rotation in range(12):
        if highestconf >= 85:
            break
        angle = rotation * 30
        #if rotation != 0:
            #print('rotating...', angle,' ', 'degrees', end = '\r')
        rotated_image = imutils.rotate(mydict[threshoption], angle = angle)
        for psm in range(6,13+1):
            config = '--oem 1 --psm %d -c tessedit_char_whitelist=123456789-' % psm
            #print('psm = {}'.format(psm))
            d = pytesseract.image_to_data(rotated_image, config = config, output_type = Output.DICT)
            #if d['text'] != ['']:
                #print('Text Detected: ',d['text'])
                #print('Confidence: ', d['conf'])
            for idx, i in enumerate(d['text']):
                if i != '' and i != '-' and '--' not in i and len(i) > 1:
                    i = remove_non_numeric(i)
                    #print('predicted label: {}, confidence: {}'.format(i, d['conf'][idx]))
                if d['conf'][idx] > (60) or is_number(str(i)):
                    if d['conf'][idx] > highestconf and len(i) == 3 or len(i) == 5:
                        highestconf = d['conf'][idx]
                        if len(i) == 3:
                            i = '-'.join(c for c in i)
                        yourtag = i
                
    return('{}***{}'.format(image,yourtag), highestconf)

In [ ]:
trainImages = os.listdir('/kaggle/working/textcropped')

fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(20, 10))

random_indices = random.sample(range(0, len(trainImages)), 20)

# Iterate through the random sample of images
for i, ax in enumerate(axes.flat):
    # Retrieve the image and preprocess it
    path =trainImages[random_indices[i]]
    path = os.path.join('/kaggle/working/textcropped/', path)
    image = cv2.imread(path)
    # Display the image in the current subplot
    ax.imshow(image)
    ax.axis('off')
# Adjust the spacing between subplots
plt.tight_layout()

# Show the figure with all subplots
plt.show()

In [ ]:
import pathlib
import pytesseract
from IPython.display import clear_output
from random import sample
from pytesseract import Output
import openpyxl

In [ ]:
from random import sample
image_list = os.listdir('/kaggle/working/textcropped')
thresholds = [130,140,150,160,170,180,190, 200,210]
threshoptions = ['graycrop','th2', 'th3', 'thotsu']
counter = 0
scored_tags = []
for picture in image_list:
    print('Progress: {}/{}'.format(counter+1, len(image_list)))
    templabel = 'INITIAL'
    highestconf = -100
    picture = os.path.join('/kaggle/working/textcropped', picture)
    img = cv2.imread(picture)
    for thresholdvalue in thresholds:
        if highestconf >= 80:
                print('final tag: ', templabel)
                break
        for thresh in threshoptions:
            if highestconf >= 80:
                break
            label, confidence = attempt_read(image = picture,
                                             threshold = thresholdvalue,
                                             highestconf = 0,
                                            threshoption = thresh)

            if confidence > highestconf and confidence != -1 and 'NO READ' not in label:
                print('replacing highest score ({}) with {}'.format(templabel.replace('.jpg', ''), label+'.jpg'))
                templabel = label
                highestconf = confidence
    if highestconf >= 83:
        scored_tags.append('{}***{}.jpg'.format(picture.split('/')[-1], templabel.split('***')[-1].replace('.jpg', '').split('/')[-1]))
    elif 'NO READ' not in templabel and 'INITIAL' not in templabel and highestconf < 85:
        scored_tags.append('{}***{}-CHECK.jpg'.format(picture.split('/')[-1], templabel.split('***')[-1].replace('.jpg', '').split('/')[-1]))
    else:
        scored_tags.append('{}***{}.jpg'.format(picture.split('/')[-1], templabel.split('***')[-1].replace('.jpg', '').split('/')[-1]))
    print('taglist: ', scored_tags)
    plt.close()

In [ ]:
source_folder1 = '/kaggle/input/example-photoset/example'

destination_folder = '/kaggle/working/Test'

os.makedirs(destination_folder, exist_ok=True)

for i, file_name in enumerate(os.listdir(source_folder1)):
    print('Progress: {}/{}'.format(i, len(os.listdir(source_folder1))), end = '\r')
    shutil.copy2(os.path.join(source_folder1, file_name), destination_folder)

In [ ]:
goodtags = 0
semitags  = 0
badtags = 0
for tag in scored_tags:
    if 'INITIAL' not in tag and 'CHECK' not in tag:
        print(tag)
        goodtags += 1
    elif 'CHECK' in tag:
        semitags +=1
    else:
        badtags += 1

print('{}% good, {}% check'.format(goodtags/len(scored_tags)*100, semitags/len(scored_tags)*100))

In [ ]:
piccounter = 0
tagcounter = 0 
for picture in sorted(os.listdir('/kaggle/working/Test')):
    if piccounter % 2 != 0:
        oldname = os.path.join('/kaggle/working/Test', picture)
        print(tagcounter)
        print(oldname)
        newname = os.path.join('/kaggle/working/Test', sorted(scored_tags)[tagcounter])
        print(newname)
        os.rename(oldname,newname)
        tagcounter += 1
    piccounter += 1
    
    
    

In [ ]:
for file in sorted(os.listdir('/kaggle/working/Test')):
    print(file)